# Pandas - Data manipulations

This notebook is the third part of the collection devoted to the pandas library.

In [ ]:
# Necesssary import evil

import jupy_helpers
import pandas as pd
import numpy as np
from IPython.display import display, HTML, IFrame

np.random.seed(42)

In [ ]:
%matplotlib inline  
# This will enable us to see plots embedded in the notebook

In [ ]:
# make the cells wide
display(HTML("<style>.container { width:95% !important; }</style>"))

In the previous notebook, we imported data from several data formats.
Let's start from there and see what basic manipulations we can do
with the DataFrames.

In [ ]:
con = 'sqlite:///./workshop_data.sqlite'

In [ ]:
# Read the data that we stored in the previous example
imdb_titles = pd.read_sql('imdb_titles', con)
imdb_ratings = pd.read_sql('imdb_ratings', con)
boxoffice = pd.read_sql('boxoffice', con)
rotten_tomatoes = pd.read_sql("rotten_tomatoes", con)

In [1]:
# alternative for Binder

# imdb_titles = pd.read_csv('../data/title.basics.tsv.gz', sep='\t', na_values="\\N"),
# imdb_ratings = pd.read_csv('../data/title.ratings.tsv.gz', sep='\t', na_values="\\N"),
# boxoffice = pd.read_csv('../data/boxoffice_march_2019.csv.gz'),
# rotten_tomatoes = pd.read_csv("../data/rotten_tomatoes_top_movies_2019-01-15.csv"),
# awards = pd.read_sql("awards", con='sqlite:///../data/awards.sqlite')

## First look at the data

Let's see what we have in the datasets...

In [ ]:
imdb_titles.head()

In [ ]:
imdb_titles.tail()

For an unknown, potentially unevenly distributed dataset, looking at the beginning / end is typically not the best idea. We'd rather sample randomly:

In [ ]:
# Show a few examples
imdb_titles.sample(20)

`describe` is often useful to get the basics statistics.

In [ ]:
imdb_titles.describe(include="all")

Notice that only some columns, to be procise columns with properly recognized numeric type, are reported.

### Column data types

In [ ]:
imdb_titles.dtypes

Let's investigate some of the columns...

**tconst** is probably the key used in the IMDB database. We will get to this later.

To analyze **titleType**, we can look at its typical values.

In [ ]:
imdb_titles.titleType.describe()

Not too many different values.

In [ ]:
imdb_titles.titleType.unique()

In [ ]:
imdb_titles.titleType.value_counts(normalize=True)

This column is a very good candidate to turn into a pandas-special, **Categorical** data type. (See https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html)

In [ ]:
imdb_titles.titleType.memory_usage()   # ~8 bytes per item

In [ ]:
title_type = imdb_titles.titleType.astype("category")
title_type.sample(10)

In [ ]:
title_type.memory_usage()              # ~1 byte per item

**Exercise:** Remember that assigning or replacing a column inside a dataframe works the same way as setting a value inside a dict. Please, replace the original "titleType" column with the new Categorical series:

In [ ]:
%exercise

... = ...

In [ ]:
%validate

assert imdb_titles['titleType'].dtype.name == 'category'

Pandas is generally quite good at guessing (inferring) number types. 
You may wonder why `startYear` is float and not int though. 
This is a consequence of not having an integer nan in numpy. There's been many discussions about this.

In [ ]:
imdb_titles.startYear.dtype

`ValueError: Cannot convert non-finite values (NA or inf) to integer` 
 is throw if you try to convert to `int`

In [ ]:
imdb_titles.startYear.astype(int)

In [ ]:
# To address that, pandas 0.24 Introduced nullable integers
imdb_titles.startYear.astype("Int64").tail(20)

### Filtering

Indexing in pandas Series / DataFrames (`[]`) support also boolean (masked) arrays. These arrays can be obtained by applying boolean operations on them.

You can also use standard **comparison operators** like `<`, `<=`, `==`, `>=`, `>`, `!=`. 

It is possible to perform **logical operators** with boolean series too. But note that `and`, `or`, `not` are keywords. You 

As an example, find all movies!

In [ ]:
is_movie = (imdb_titles.titleType == "movie")
is_movie.sample(10)

Now we can directly apply the boolean mask. (Note: This is no magic. You can construct the mask yourself)

In [ ]:
imdb_titles[is_movie].sample(10)

Or maybe we should include TV movies as well?

In [ ]:
# isin takes an array of possible values
movie_titles = imdb_titles[imdb_titles.titleType.isin(["movie", "tvMovie"])]
movie_titles.sample(10)

We may wonder why we have two title-ish columns: originalTitle and primaryTitle.

In [ ]:
different_title = (movie_titles.originalTitle != movie_titles.primaryTitle)
different_title.name = "Different title"   # Series can have names (Note: DataFrames can't)
different_title.value_counts()

### Visualization intermezzo

Without much further ado, let's create our first plot.

In [ ]:
different_title.value_counts().plot(kind="bar")

**Exercise:** Display some random movies that have different original and primary titles.

In [ ]:
%exercise

movie_titles[___].___

We may also want to get rid of wrong / incomplete data.

In [ ]:
# Movies from the future?
imdb_titles[imdb_titles.startYear > 2019].startYear.value_counts()

In [ ]:
# Movies with startYear undefined
imdb_titles[imdb_titles.startYear.isna()].head()

**Exercise:** Use the `imdb_titles.startYear > 2019` filter and `dropna` method to get rid of rows with wrong / undefined years.

In [ ]:
%exercise

imdb_correct_titles = imdb_titles[___].dropna(subset=___)

In [ ]:
%validate

assert not (imdb_correct_titles.startYear > 2019).any()
assert not imdb_correct_titles.startYear.isna().any()
assert imdb_correct_titles.shape == (1694587, 9)

**Exercise:** Drop `titleType` and `endYear` columns and rows with `NaN` values (use `dropna`).

In [ ]:
%exercise

movie_titles = (movie_titles
                .drop(columns=___)
                .___
                )

In [ ]:
%validate

assert not any(result.isna().any())
assert sorted(result.columns) == ['genres', 'isAdult', 'originalTitle',  'primaryTitle', 
                                        'runtimeMinutes', 'startYear', 'tconst']

## Sorting

In [ ]:
# Display 5 longest movies 
movie_titles.sort_values("runtimeMinutes", ascending=False).head()

Hmm, let's look at *Modern Times Forever* - an apt title :)

In [ ]:
IFrame(
    src="https://en.wikipedia.org/wiki/Modern_Times_Forever_(Stora_Enso_Building,_Helsinki)",
    width=900,
    height=400,
)

In [ ]:
# Alternative
movie_titles.nlargest(5, "runtimeMinutes")

**Exercise:** Find the 10 oldest movies that are longer than 2 hours

In [ ]:
%exercise

result = movie_titles[___].___(___, ___)
result

In [ ]:
%validate

assert result.iloc[9]["startYear"] == 1914

In [ ]:
movie_titles[(movie_titles["primaryTitle"].str.startswith("Indiana Jones")) & (movie_titles["titleType"] == "movie")]

## Arithmetics and string manipulation

Standard **arithmetic operators** work on numerical columms too. And so do mathematical functions. Note all such operations are performed in a vector-like fashion.

In [ ]:
movie_titles[["originalTitle", "startYear"]].assign(age = 2019 - movie_titles["startYear"]).sample(20)

**Exercise:** Calculate the length of movies in hours.

In [ ]:
%exercise

result = movie_titles[["originalTitle", "runtimeMinutes"]].assign(... = ...)
result.sample(10)

In [ ]:
%validate

assert result.iloc[0]["hours"] == 0.75

In [ ]:
# If only the following had any sense!
np.sin(movie_titles["runtimeMinutes"]).sample(10)

### Summary statistics

Note that the following methods return scalars

In [ ]:
# Average length of the movies
movie_titles["runtimeMinutes"].mean()

In [ ]:
# Standard deviation
movie_titles["runtimeMinutes"].std()

In [ ]:
movie_titles["startYear"].max()

**Exercise:** How many years would it take to watch all movies in the catalogue?

In [ ]:
%exercise
result = movie_titles["runtimeMinutes"] ...
result

In [ ]:
%validate

assert np.round(result, 3) == 65.154

### Basic string operations

These are typically accessed using the `.str` "accessor" of the Series like this:
    
- series.str.lower
- series.str.split
- series.str.startswith
- series.str.contains
- ...

In [ ]:
movie_titles[(movie_titles["primaryTitle"].str.startswith("Star Wars: Episode")) & (movie_titles["titleType"] == "movie")]

**Exercise:** Find all Pink Panther movies. Note that their title does not necessarily start with "Pink"

In [ ]:
%exercise

is_pink = ...
pink = movie_titles[is_pink]
pink

In [ ]:
%validate

assert pink.shape[0] == 14

In [ ]:
# String arithmetics work too!
url = "https://www.imdb.com/title/" + movie_titles["tconst"]
movie_titles[["primaryTitle"]].assign(url=url).sample(10)

Let's investigate the genres a bit:

In [ ]:
split_genres = movie_titles.genres.str.split(",").dropna()
split_genres.sample(10)

In [ ]:
genres = {genre for row in split_genres.iteritems() for genre in row[1]}

In [ ]:
genres

In [ ]:
# "One-hot" encoding of genres
movie_with_genres = movie_titles[['primaryTitle']].assign(
    **{genre : movie_titles.genres.str.contains(genre) * 1 for genre in genres}
)
movie_with_genres.sample(10)

In [ ]:
genre_counts = movie_with_genres[movie_with_genres == 1].iloc[:,1:].count()
genre_counts

In [ ]:
genre_counts.plot(kind="bar")

In [ ]:
movie_titles["startYear"].plot(kind="hist")

We see the one outlier and in general, the plots are not so nice. Being explicit sometimes helps.

In [ ]:
ax = movie_titles["startYear"].plot(kind="hist", bins=23, range=(1890, 2020))

No movies before 1900 / 1910? Really? Let's check:

In [ ]:
ax = movie_titles["startYear"].plot(kind="hist", bins=23, range=(1890, 2020))
ax.set_yscale("log")

**Exercise:** Plot a histogram of runtime minutes (limiting to a reasonable range).

In [ ]:
%exercise

movie_titles[___].plot(___)

In [ ]:
# Note: this is something that could be mor easily done with .groupby operation
average_genre_length = (movie_with_genres.iloc[:,1:].apply(lambda s: s * movie_titles["runtimeMinutes"])).sum() / genre_counts

In [ ]:
average_genre_length

**Exercise:** Sort the genres from the longest to shortest and visualize them using a bar plot.

In [ ]:
%exercise

___.___.plot(___)

**Exercise**: Find proper filters for the Lord of the Rings trilogy. Note that you will probably need to combine more conditions (multiple solutions possible).

In [ ]:
%exercise

is_lotr = ...
is_real_lotr = is_lotr & 

movie_titles[is_real_lotr]

**Exercise**: What is longer? The Lord of the Rings trilogy or The Hobbit "trilogy"?

Use the previous as a base to find the Hobbit movies. Then calculate the total length of each trio.

In [ ]:
%exercise

is_hobbit = ...
is_real_hobbit = ...

hobbit_length = ...
lotr_length = ...

print(f"The Hobbit length: {hobbit_length}, The Lord of the Rings length: {lotr_length}")

pd.concat([movie_titles[is_real_lotr], movie_titles[is_real_hobbit]]).sort_values("startYear")

In [ ]:
%validate

assert hobbit_length % 100 == 74
assert lotr_length // 100 == 5

**Exercise:** Find the movie with the (primary) longest title.

Hint: `idxmax()` method on the Series returns the index of the item with the maximum value. You can't (at least should) not use the maximum value itself.

In [ ]:
%exercise

longest_title_movie = ...

print(longest_title_movie["primaryTitle"])

longest_title_movie

In [ ]:
%validate

assert longest_title_movie["primaryTitle"][2] == "s"